In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1355.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1642.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_9.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1473.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1639.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1928.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_829.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_611.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_1314.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/t-shirt/IMG_486.jpg
/kaggle/input/an

/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_1676.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_88.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_609.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_799.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_1294.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_1519.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_1491.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_200.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bus/IMG_2014.jpg
/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training/school-bu

In [29]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [30]:
import os
import tensorflow as tf
import numpy as np

SEED = 1234
tf.random.set_seed(SEED)  

cwd = os.getcwd()
print(cwd)

/kaggle/working


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

test_data_gen = ImageDataGenerator(rescale = 1./255)

In [32]:
dataset_dir = os.path.join('../input/ann-and-dl-image-classification/', 'Classification_Dataset')
bs = 8

img_h = 256
img_w = 256

num_classes = 20

classList = ['owl', 
             'galaxy', 
             'lightning', 
             'wine-bottle',
             't-shirt',
             'waterfall',
             'sword',
             'school-bus',
             'calculator',
             'sheet-music',
             'airplanes',
             'lightbulb',
             'skyscraper',
             'mountain-bike',
             'fireworks',
             'computer-monitor',
             'bear',
             'grand-piano',
             'kangaroo',
             'laptop']

training_dir = os.path.join(dataset_dir, 'training')

train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classList,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='training' # set as training data
                                              )

valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classList,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               subset='validation' # set as validation data
                                              )

test_dir = os.path.join(dataset_dir, 'test')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             classes=classList,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

Found 1247 images belonging to 20 classes.
Found 307 images belonging to 20 classes.
Found 0 images belonging to 20 classes.


In [33]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                                output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

test_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                              output_types=(tf.float32, tf.float32),
                                                output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

test_dataset = test_dataset.repeat()
train_gen.class_indices

{'owl': 0,
 'galaxy': 1,
 'lightning': 2,
 'wine-bottle': 3,
 't-shirt': 4,
 'waterfall': 5,
 'sword': 6,
 'school-bus': 7,
 'calculator': 8,
 'sheet-music': 9,
 'airplanes': 10,
 'lightbulb': 11,
 'skyscraper': 12,
 'mountain-bike': 13,
 'fireworks': 14,
 'computer-monitor': 15,
 'bear': 16,
 'grand-piano': 17,
 'kangaroo': 18,
 'laptop': 19}

In [34]:
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()  # we can specify the activation function directly in Conv2D
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [35]:
depth = 5
start_f = 8
num_classes = 20

class CNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x
    
# Create Model instance
model = CNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

In [36]:
# Visualize created model as a table
model.feature_extractor.summary()

# Visualize initialized weights
model.weights

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block_10 (ConvBlock)    multiple                  224       
_________________________________________________________________
conv_block_11 (ConvBlock)    multiple                  1168      
_________________________________________________________________
conv_block_12 (ConvBlock)    multiple                  4640      
_________________________________________________________________
conv_block_13 (ConvBlock)    multiple                  18496     
_________________________________________________________________
conv_block_14 (ConvBlock)    multiple                  73856     
Total params: 98,384
Trainable params: 98,384
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'sequential_4/conv_block_10/conv2d_10/kernel:0' shape=(3, 3, 3, 8) dtype=float32, numpy=
 array([[[[ 1.87292695e-02, -6.66571707e-02,  4.01918888e-02,
           -1.26130164e-01, -3.43003571e-02,  1.99994743e-01,
            3.55525315e-03, -6.95133060e-02],
          [ 1.37295932e-01,  4.19998467e-02,  1.32354498e-01,
           -8.24365020e-03,  4.30276692e-02, -2.35006183e-01,
           -3.16811651e-02,  2.35468864e-01],
          [ 2.36862689e-01, -1.74060702e-01,  2.43510842e-01,
            1.38533324e-01, -1.37798309e-01, -5.41950762e-03,
            6.55457079e-02,  3.13819945e-02]],
 
         [[-4.74313945e-02, -4.98768538e-02,  8.99522901e-02,
            1.34971857e-01, -2.68238485e-02, -2.18192115e-01,
           -1.25084400e-01, -9.58562642e-02],
          [-8.99255723e-02, -1.51148498e-01,  1.60095960e-01,
            4.54548597e-02, -6.18434399e-02,  1.69046938e-01,
           -1.27805948e-01,  1.58601552e-01],
          [-1.66178837e-01, -1.14610642e-01,

In [37]:
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [38]:
import os
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
            
# Early Stopping
callbacks = []

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)
    
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen)
          )

Train for 156 steps, validate for 39 steps
Epoch 1/40
 85/156 [===============>..............] - ETA: 26s - loss: 3.0003 - accuracy: 0.0446

KeyboardInterrupt: 

In [ ]:
from PIL import Image
# ....
path = '../input/ann-and-dl-image-classification/Classification_Dataset/'
image_filenames = next(os.walk(path + 'test'))[2]

results = {}
for image_name in image_filenames:
    img = Image.open(path + 'test/' + image_name).convert('RGB')
    img_array = np.array(img) 
    
    img_array = np.resize(img_array, [img_h, img_w, 3]) #For a RGB image
    img_array = np.expand_dims(img_array, 0) 
   
    img_array = img_array/255;
   

    prediction = model.predict(img_array)
    prediction = np.argmax(prediction, axis= -1)   # predicted class #controlla axis = 0 o usa squeeze
    
    results[image_name] = prediction
    
    
    print(results[image_name])
    

create_csv(results)